In [ ]:
import os, re, gc, time, math, random, warnings
from typing import Optional, List, Tuple
from collections import Counter
warnings.filterwarnings('ignore')
print("Cell 1 OK")

In [ ]:
import numpy as np
import torch
np.random.seed(42)
random.seed(42)
torch.manual_seed(42)
print(f"torch {torch.__version__} | CUDA {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"{torch.cuda.get_device_name(0)} | {torch.cuda.get_device_properties(0).total_memory/1024**3:.1f}GB")

In [ ]:
try:
    import bitsandbytes as bnb
    BNB_OK = True
    print(f"bitsandbytes {bnb.__version__}")
except:
    BNB_OK = False
    print("bitsandbytes: NO")

In [ ]:
QWEN_PATH = "/kaggle/input/qwen-72b-math-nf4"
TEST_PATH = "/kaggle/input/ai-mathematical-olympiad-progress-prize-3/test.csv"

MEM_CEIL = 70.0
SAMPLES = 6
MAX_TOK = 1024
TEMPS = [0.3, 0.5, 0.6, 0.7, 0.8, 0.9]

def gpu_mem(): return torch.cuda.memory_allocated()/1024**3 if torch.cuda.is_available() else 0
def mem_ok(): return gpu_mem() < MEM_CEIL
def force_gc():
    gc.collect()
    if torch.cuda.is_available(): torch.cuda.empty_cache(); torch.cuda.synchronize()

print(f"Qwen: {os.path.exists(QWEN_PATH)}")
print(f"Test: {os.path.exists(TEST_PATH)}")
if os.path.exists(QWEN_PATH):
    files = os.listdir(QWEN_PATH)
    sz = sum(os.path.getsize(os.path.join(QWEN_PATH,f)) for f in files if os.path.isfile(os.path.join(QWEN_PATH,f)))
    print(f"Qwen: {len(files)} files, {sz/1024**3:.1f}GB")

In [ ]:
def try_symbolic(p: str) -> Optional[int]:
    try:
        t, o = p.lower(), p
        m = re.search(r'(\d+)\^(\d+)\s*mod\s*(\d+)', o)
        if m: return pow(int(m.group(1)), int(m.group(2)), int(m.group(3)))
        m = re.search(r'(\d+)\s*[\^\*]{1,2}\s*(\d+).*?(?:mod|modulo|divided by)\s*(\d+)', t)
        if m: return pow(int(m.group(1)), int(m.group(2)), int(m.group(3)))
        m = re.search(r'(\d+)\s*(?:mod|modulo)\s*(\d+)', t)
        if m: return int(m.group(1)) % int(m.group(2))
        m = re.search(r'remainder.*?(\d+).*?divided.*?(\d+)', t)
        if m: return int(m.group(1)) % int(m.group(2))
        m = re.search(r'(\d+)\s*[×\*]\s*(\d+)', o)
        if m:
            r = int(m.group(1)) * int(m.group(2))
            if 0 <= r < 100000: return r
        m = re.search(r'(\d+)\s*times\s*(\d+)', t)
        if m:
            r = int(m.group(1)) * int(m.group(2))
            if 0 <= r < 100000: return r
        m = re.search(r'sum.*?divisors.*?(\d+)', t)
        if m:
            n = int(m.group(1))
            if n < 10000: return sum(i for i in range(1,n+1) if n%i==0)
        m = re.search(r'(?:gcd|greatest common divisor).*?(\d+).*?(\d+)', t)
        if m: return math.gcd(int(m.group(1)), int(m.group(2)))
    except: pass
    return None

assert try_symbolic("2^10 mod 7") == 2
print("symbolic OK")

In [ ]:
def clean(t):
    if not t: return ""
    t = re.sub(r'Study\.com.*', '', t, flags=re.DOTALL|re.I)
    t = re.sub(r'Become a.*?member.*', '', t, flags=re.I)
    return t.strip()

def extract(t) -> Optional[int]:
    t = clean(t)
    if not t: return None
    for p in [r'\\boxed\{(\d+)\}', r'boxed\{(\d+)\}', r'\*\*(\d+)\*\*\s*$', r'[Aa]nswer[^0-9]*(\d+)', r'=\s*(\d+)\s*$']:
        m = re.findall(p, t, re.M|re.I)
        if m:
            try:
                v = int(m[-1])
                if 0 <= v < 100000: return v
            except: pass
    for s in reversed(re.findall(r'\b(\d+)\b', t[-300:])):
        try:
            v = int(s)
            if 1 < v < 100000: return v
        except: pass
    return None

print("extract OK")

In [ ]:
def cic(samples: List[int]) -> Tuple[int, float]:
    if not samples: return 0, 0.0
    if len(samples) == 1: return samples[0], 0.5
    c = Counter(samples)
    top = c.most_common()
    best, cnt = top[0]
    conf = min(0.95, cnt / len(samples))
    if len(top) > 1 and top[0][1] == top[1][1]:
        best = int(np.median([x for x,_ in top[:2]]))
    return max(0, min(99999, int(best))), conf

print("cic OK")

In [ ]:
class QwenModel:
    def __init__(self):
        self.model = None
        self.tok = None
        self.loaded = False
        
    def load(self):
        if self.loaded: return True
        if not os.path.exists(QWEN_PATH):
            print("Qwen: NOT FOUND")
            return False
        try:
            from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
            print("Loading Qwen...")
            
            # Try direct first
            try:
                self.model = AutoModelForCausalLM.from_pretrained(
                    QWEN_PATH, device_map="auto", trust_remote_code=True, low_cpu_mem_usage=True
                )
                print(f"  Direct load: {gpu_mem():.1f}GB")
            except Exception as e:
                print(f"  Direct failed: {e}")
                if BNB_OK:
                    print("  Trying explicit NF4...")
                    cfg = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4",
                                            bnb_4bit_compute_dtype=torch.float16, bnb_4bit_use_double_quant=True)
                    self.model = AutoModelForCausalLM.from_pretrained(
                        QWEN_PATH, quantization_config=cfg, device_map="auto", trust_remote_code=True, low_cpu_mem_usage=True
                    )
                    print(f"  NF4 load: {gpu_mem():.1f}GB")
                else:
                    print("  Trying FP16...")
                    self.model = AutoModelForCausalLM.from_pretrained(
                        QWEN_PATH, torch_dtype=torch.float16, device_map="auto", trust_remote_code=True, low_cpu_mem_usage=True
                    )
                    print(f"  FP16 load: {gpu_mem():.1f}GB")
            
            self.tok = AutoTokenizer.from_pretrained(QWEN_PATH, trust_remote_code=True)
            if self.tok.pad_token is None: self.tok.pad_token = self.tok.eos_token
            self.loaded = True
            print(f"Qwen ready: {gpu_mem():.1f}GB")
            return True
        except Exception as e:
            print(f"Qwen FAILED: {e}")
            import traceback; traceback.print_exc()
            return False
    
    def gen(self, prompt: str, temp: float) -> str:
        if not self.loaded: return ""
        try:
            inp = self.tok(prompt, return_tensors="pt", truncation=True, max_length=512).to(self.model.device)
            with torch.no_grad():
                out = self.model.generate(**inp, max_new_tokens=MAX_TOK, temperature=temp, do_sample=True, top_p=0.9, pad_token_id=self.tok.pad_token_id)
            return self.tok.decode(out[0][inp["input_ids"].shape[1]:], skip_special_tokens=True)
        except Exception as e:
            print(f"  gen err: {e}")
            return ""

print("QwenModel OK")

In [ ]:
PROMPT = """Solve step by step. Be precise.

PROBLEM_HERE

Final answer as \\boxed{{INTEGER}} (0-99999)."""

class Engine:
    def __init__(self):
        self.qwen = QwenModel()
        self.times = []
        
    def solve(self, problem: str, pid: str) -> int:
        t0 = time.time()
        print(f"\n[{pid}] {problem[:60]}...")
        
        # Symbolic
        sym = try_symbolic(problem)
        if sym is not None:
            print(f"  SYM: {sym}")
            self.times.append(time.time()-t0)
            return sym
        
        # LLM
        if not self.qwen.load():
            print("  MODEL FAILED - returning 0")
            return 0
        
        prompt = PROMPT.replace("PROBLEM_HERE", problem)
        answers = []
        for temp in TEMPS[:SAMPLES]:
            resp = self.qwen.gen(prompt, temp)
            ans = extract(resp)
            if ans is not None:
                answers.append(ans)
                print(f"    T{temp}: {ans}")
            else:
                print(f"    T{temp}: -")
        
        answer, conf = cic(answers)
        elapsed = time.time() - t0
        self.times.append(elapsed)
        print(f"  >>> {answer} | conf={conf:.2f} | {elapsed:.1f}s")
        return answer

print("Engine OK")

In [ ]:
def run_comp():
    print("="*50)
    print("AIMO3 V12.9 QWEN-ONLY")
    print("="*50)
    eng = Engine()
    def predict(pid, prob):
        try: return eng.solve(prob, pid)
        except Exception as e:
            print(f"ERR: {e}")
            return 0
    from kaggle_evaluation import aimo_2_inference_server as srv
    server = srv.AIMO2InferenceServer(predict)
    if os.getenv("KAGGLE_IS_COMPETITION_RERUN"): server.serve()
    else: server.run_local_gateway((TEST_PATH,))

def run_local():
    print("="*50)
    print("AIMO3 V12.9 LOCAL")
    print("="*50)
    eng = Engine()
    tests = [
        ("t1", "2^10 mod 7", 2),
        ("t2", "17 * 23", 391),
        ("t3", "sum of divisors of 28", 56),
        ("t4", "If f(x)=x^2-3x+5, what is f(4)?", 9),
    ]
    ok = 0
    for pid, p, exp in tests:
        r = eng.solve(p, pid)
        if r == exp: ok += 1
        print(f"  {pid}: {r} {'OK' if r==exp else f'WRONG({exp})'}")
    print(f"\n{ok}/{len(tests)}")
    if eng.times: print(f"Avg: {np.mean(eng.times):.1f}s")

print("runners OK")

In [ ]:
print(f"GPU: {torch.cuda.get_device_properties(0).total_memory/1024**3:.1f}GB" if torch.cuda.is_available() else "No GPU")
try:
    from kaggle_evaluation import aimo_2_inference_server
    print("kaggle_eval: YES")
    HAS = True
except:
    print("kaggle_eval: NO")
    HAS = False

if os.path.exists(TEST_PATH) and HAS: run_comp()
else: run_local()